In [1]:
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd

In [2]:
"""
11 : 서울 : 2074
21 : 부산 : 554
22 : 대구 : 304
23 : 인천 : 288
24 : 광주 : 165
25 : 대전 : 236
26 : 울산 : 154
45 : 세종 : 62
31 : 경기 : 1275
32 : 강원 : 750
33 : 충북 : 953
34 : 충남 : 1261
35 : 전북 : 1079
36 : 전남 : 1539
37 : 경북 : 2463
38 : 경남 : 2435
50 : 제주 : 440
ZZ : 전국일원 : 131
"""
total_index = [11, 21, 22, 23, 24, 25, 26, 45, 31, 32, 33, 34, 35, 36, 37, 38, 50] #'ZZ' 일단 제외
total_name = {11 : '서울', 21 : '부산', 22 : '대구', 
            23 : '인천', 24 : '광주', 25 : '대전',
            26 : '울산', 45 : '세종', 31 : '경기', 
            32 : '강원', 33 : '충북', 34 : '충남',
            35 : '전북', 36 : '전남', 37 : '경북',
            38 : '경남', 50 : '제주'} #'ZZ' : '전국일원' 일단 제외
total_num = {11 : 2074, 21 : 554, 22 : 304, 
            23 : 288, 24 : 165, 25 : 236,
            26 : 154, 45 : 62, 31 : 1275, 
            32 : 750, 33 : 953, 34 : 1261,
            35 : 1079, 36 : 1539, 37 : 2463,
            38 : 2435, 50 : 440} #'ZZ' : 131 일단 제외

In [3]:
def get_info(index):
    
    df = pd.DataFrame(columns=['문화재명','시군구명','종목','경도','위도','대표이미지'])
    
    index_name = str(total_name[index])
    index_num = int(total_num[index])
    
    for i in range(index_num):

        default_url='https://www.cha.go.kr/cha/SearchKindOpenapiList.do?pageUnit=1&ccbaCtcd='+str(index)
        loop_url = default_url + '&pageIndex=' + str(i+1)

        url_01 = requests.get(loop_url)
        soup_01 = bs4(url_01.text, 'html.parser')

        # 문화재명
        name = str(soup_01.select('ccbaMnm1'))
        name = name.replace("[<ccbamnm1><![CDATA[","")
        name = name.replace("]]></ccbamnm1>]","")

        # 시군구명
        county = str(soup_01.select('ccsiName'))
        county = county.replace("[<ccsiname><![CDATA[","")
        county = county.replace("]]></ccsiname>]","")

        # 종목코드
        code = str(soup_01.select('ccbaKdcd'))
        code = code.replace("[<ccbakdcd>","")
        code = code.replace("</ccbakdcd>]","")
        code_int = int(code)

        # 종목이름(한글)
        code_type = {11 : '국보', 12 : '보물', 13 : '사적', 
                    14 : '사적및명승', 15 : '명승', 16 : '천연기념물',
                    17 : '국가무형문화재', 18 : '국가민속문화재', 21 : '시도유형문화재', 
                    22 : '시도무형문화재', 23 : '시도기념물', 24 : '시도민속문화재',
                    31 : '문화재자료', 79 : '등록문화재', 80 : '이북5도 무형문화재',
                    25 : 'ERROR'} #한강대교 1749 25 에러
        code_hangeul = code_type[int(code)]

        # 시도코드
        city_code = str(soup_01.select('ccbaCtcd'))
        city_code = city_code.replace("[<ccbactcd>","")
        city_code = city_code.replace("</ccbactcd>]","")

        # 문화재지정번호
        num = str(soup_01.select('ccbaAsno'))
        num = num.replace("[<ccbaasno>","")
        num = num.replace("</ccbaasno>]","")
        num_int = int(num)

        # 경도
        longitude = str(soup_01.select('longitude'))
        longitude = longitude.replace("[<longitude>","")
        longitude = longitude.replace("</longitude>]","")
        longitude_float = float(longitude)

        # 위도
        latitude = str(soup_01.select('latitude'))
        latitude = latitude.replace("[<latitude>","")
        latitude = latitude.replace("</latitude>]","")
        latitude_float = float(latitude)

        # 대표이미지
        image_url = 'https://www.cha.go.kr/cha/SearchImageOpenapi.do?ccbaKdcd='+code+'&ccbaCtcd='+city_code+'&ccbaAsno='+num

        url_02 = requests.get(image_url)
        soup_02 = bs4(url_02.text, 'html.parser')

        image_temp = str(soup_02.select('imageUrl'))
        image_temp_partition = image_temp.partition('</imageurl>')
        image_link = image_temp_partition[0].replace('[<imageurl>',"")

        # 위도, 경도 정보없는 문화재 제거
        if longitude_float == 0 or latitude_float == 0 or code_int == 17 or code_int == 22 or code_int == 25:
            continue
        
        df.loc[i]=[name,county,code_hangeul,longitude,latitude,image_link]

        #print(name, ' ', i+1, 'page completed')

    temp_csv = './csv_list/'+index_name+'.csv'
    df.to_csv(temp_csv, index=False)
    print(index_name+" 조회.")
    
    return df

In [4]:
for i in range(len(total_index)):
    temp_total_index = total_index[i]
    total_count = get_info(temp_total_index)
    print(str(len(total_count))+"(개)")

서울 조회.
1330(개)
부산 조회.
281(개)
대구 조회.
189(개)
인천 조회.
211(개)
광주 조회.
126(개)
대전 조회.
163(개)
울산 조회.
107(개)
세종 조회.
27(개)
경기 조회.
897(개)
강원 조회.
614(개)
충북 조회.
727(개)
충남 조회.
977(개)
전북 조회.
786(개)
전남 조회.
1274(개)
경북 조회.
1964(개)
경남 조회.
1778(개)
제주 조회.
369(개)


In [6]:
#ZZ : 131 : 전국일원

df = pd.DataFrame(columns=['문화재명','시군구명','종목','경도','위도','대표이미지'])

for i in range(131):
    
    default_url='https://www.cha.go.kr/cha/SearchKindOpenapiList.do?pageUnit=1&ccbaCtcd=ZZ'
    loop_url = default_url + '&pageIndex=' + str(i+1)

    url_01 = requests.get(loop_url)
    soup_01 = bs4(url_01.text, 'html.parser')

    # 문화재명
    name = str(soup_01.select('ccbaMnm1'))
    name = name.replace("[<ccbamnm1><![CDATA[","")
    name = name.replace("]]></ccbamnm1>]","")

    # 시군구명
    county = str(soup_01.select('ccsiName'))
    county = county.replace("[<ccsiname><![CDATA[","")
    county = county.replace("]]></ccsiname>]","")

    # 종목코드
    code = str(soup_01.select('ccbaKdcd'))
    code = code.replace("[<ccbakdcd>","")
    code = code.replace("</ccbakdcd>]","")
    code_int = int(code)

    # 종목이름(한글)
    code_type = {11 : '국보', 12 : '보물', 13 : '사적', 
                14 : '사적및명승', 15 : '명승', 16 : '천연기념물',
                17 : '국가무형문화재', 18 : '국가민속문화재', 21 : '시도유형문화재', 
                22 : '시도무형문화재', 23 : '시도기념물', 24 : '시도민속문화재',
                31 : '문화재자료', 79 : '등록문화재', 80 : '이북5도 무형문화재',
                25 : 'ERROR'} #한강대교 1749 25 에러
    code_hangeul = code_type[int(code)]

    # 시도코드
    city_code = str(soup_01.select('ccbaCtcd'))
    city_code = city_code.replace("[<ccbactcd>","")
    city_code = city_code.replace("</ccbactcd>]","")

    # 문화재지정번호
    num = str(soup_01.select('ccbaAsno'))
    num = num.replace("[<ccbaasno>","")
    num = num.replace("</ccbaasno>]","")
    num_int = int(num)

    # 경도
    longitude = str(soup_01.select('longitude'))
    longitude = longitude.replace("[<longitude>","")
    longitude = longitude.replace("</longitude>]","")
    longitude_float = float(longitude)

    # 위도
    latitude = str(soup_01.select('latitude'))
    latitude = latitude.replace("[<latitude>","")
    latitude = latitude.replace("</latitude>]","")
    latitude_float = float(latitude)

    # 대표이미지
    image_url = 'https://www.cha.go.kr/cha/SearchImageOpenapi.do?ccbaKdcd='+code+'&ccbaCtcd='+city_code+'&ccbaAsno='+num

    url_02 = requests.get(image_url)
    soup_02 = bs4(url_02.text, 'html.parser')

    image_temp = str(soup_02.select('imageUrl'))
    image_temp_partition = image_temp.partition('</imageurl>')
    image_link = image_temp_partition[0].replace('[<imageurl>',"")

    # 위도, 경도 정보없는 문화재 제거
    if longitude_float == 0 or latitude_float == 0 or code_int == 17 or code_int == 22 or code_int == 25:
        continue
    
    df.loc[i]=[name,county,code_hangeul,longitude,latitude,image_link]

df.to_csv('./csv_list/전국일원.csv', index=False)
print("전국일원 조회.")
print(str(len(df))+"(개)")

    #print(name, ' ', i+1, 'page completed')

전국일원 조회.
1(개)
